<a href="https://colab.research.google.com/github/eagle034/ray_tracer_colab_practice/blob/main/Copy_of_Ray_Tracer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ray Tracer in Colab

This is a Colab setup to build and run a C++ ray tracer implementation forked from https://raytracing.github.io/. A thin Python/Cython layer is created to run the C++ code, and display the rendered image in the notebook. This allows both the high performance from C++ code and the flexibility of Python/Colab, but at the cost of requiring some Cython code to interface between them.

## Directory structure

The C++ sources are stored in the [`ray_tracer_colab`](https://github.com/artoowang/ray_tracer_colab) repo, and it will be checked out to the working directory of the Colab runtime in the initialization cell below.

When running the Colab runtime locally, it is also possible to manually check out the `ray_tracer_colab` repo under exactly that name. If such directory exists, initialization will use that directory and skip the checking out. This allows changing the C++ code locally (e.g., in a terminal) and rebuild the Cython module in this notebook to pick up the changes, which is useful for development.

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Initialization

In [2]:
import os
if os.path.exists('ray_tracer_colab'):
  print('Use local ray_tracer_colab repo.')
else:
  print('Checking out ray_tracer_colab repo ...')
  !git clone https://github.com/artoowang/ray_tracer_colab.git

%ls

Checking out ray_tracer_colab repo ...
Cloning into 'ray_tracer_colab'...
remote: Enumerating objects: 106, done.
remote: Total 106 (delta 0), reused 0 (delta 0), pack-reused 106
Receiving objects: 100% (106/106), 2.65 MiB | 31.97 MiB/s, done.
Resolving deltas: 100% (45/45), done.
gdrive/  ray_tracer_colab/  sample_data/


In [3]:
import os
if os.path.exists('raytracing.github.io'):
  print('Use local raytracing github repo.')
else:
  print('Checking out raytracing github repo ...')
  !git clone https://github.com/RayTracing/raytracing.github.io.git

%ls

Checking out raytracing github repo ...
Cloning into 'raytracing.github.io'...
remote: Enumerating objects: 8521, done.
remote: Counting objects: 100% (696/696), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 8521 (delta 467), reused 600 (delta 426), pack-reused 7825
Receiving objects: 100% (8521/8521), 19.36 MiB | 39.65 MiB/s, done.
Resolving deltas: 100% (6241/6241), done.
gdrive/  ray_tracer_colab/  raytracing.github.io/  sample_data/


# RT TheNextWeek

In [30]:
%%file main.cc

//==============================================================================================
// Originally written in 2016 by Peter Shirley <ptrshrl@gmail.com>
//
// To the extent possible under law, the author(s) have dedicated all copyright and related and
// neighboring rights to this software to the public domain worldwide. This software is
// distributed without any warranty.
//
// You should have received a copy (see file COPYING.txt) of the CC0 Public Domain Dedication
// along with this software. If not, see <http://creativecommons.org/publicdomain/zero/1.0/>.
//==============================================================================================

#include "rtweekend.h"
#include "rtw_stb_image.h"
#include "box.h"
#include "bvh.h"
#include "camera.h"
//#include "color.h"
#include "constant_medium.h"
#include "hittable_list.h"
#include "material.h"
#include "moving_sphere.h"
#include "sphere.h"
#include "texture.h"

#include <iostream>
#define CHANNEL_NUM 3

color ray_color(const ray& r, const color& background, const hittable& world, int depth) {
    hit_record rec;

    // If we've exceeded the ray bounce limit, no more light is gathered.
    if (depth <= 0)
        return color(0,0,0);

    // If the ray hits nothing, return the background color.
    if (!world.hit(r, 0.001, infinity, rec))
        return background;

    ray scattered;
    color attenuation;
    color emitted = rec.mat_ptr->emitted(rec.u, rec.v, rec.p);

    if (!rec.mat_ptr->scatter(r, rec, attenuation, scattered))
        return emitted;

    return emitted + attenuation * ray_color(scattered, background, world, depth-1);
}


hittable_list random_scene() {
    hittable_list world;

    auto checker = make_shared<checker_texture>(color(0.2, 0.3, 0.1), color(0.9, 0.9, 0.9));

    world.add(make_shared<sphere>(point3(0,-1000,0), 1000, make_shared<lambertian>(checker)));

    for (int a = -11; a < 11; a++) {
        for (int b = -11; b < 11; b++) {
            auto choose_mat = random_double();
            point3 center(a + 0.9*random_double(), 0.2, b + 0.9*random_double());

            if ((center - vec3(4, 0.2, 0)).length() > 0.9) {
                shared_ptr<material> sphere_material;

                if (choose_mat < 0.8) {
                    // diffuse
                    auto albedo = color::random() * color::random();
                    sphere_material = make_shared<lambertian>(albedo);
                    auto center2 = center + vec3(0, random_double(0,.5), 0);
                    world.add(make_shared<moving_sphere>(
                        center, center2, 0.0, 1.0, 0.2, sphere_material));
                } else if (choose_mat < 0.95) {
                    // metal
                    auto albedo = color::random(0.5, 1);
                    auto fuzz = random_double(0, 0.5);
                    sphere_material = make_shared<metal>(albedo, fuzz);
                    world.add(make_shared<sphere>(center, 0.2, sphere_material));
                } else {
                    // glass
                    sphere_material = make_shared<dielectric>(1.5);
                    world.add(make_shared<sphere>(center, 0.2, sphere_material));
                }
            }
        }
    }

    auto material1 = make_shared<dielectric>(1.5);
    world.add(make_shared<sphere>(point3(0, 1, 0), 1.0, material1));

    auto material2 = make_shared<lambertian>(color(0.4, 0.2, 0.1));
    world.add(make_shared<sphere>(point3(-4, 1, 0), 1.0, material2));

    auto material3 = make_shared<metal>(color(0.7, 0.6, 0.5), 0.0);
    world.add(make_shared<sphere>(point3(4, 1, 0), 1.0, material3));

    return hittable_list(make_shared<bvh_node>(world, 0.0, 1.0));
}


hittable_list two_spheres() {
    hittable_list objects;

    auto checker = make_shared<checker_texture>(color(0.2, 0.3, 0.1), color(0.9, 0.9, 0.9));

    objects.add(make_shared<sphere>(point3(0,-10, 0), 10, make_shared<lambertian>(checker)));
    objects.add(make_shared<sphere>(point3(0, 10, 0), 10, make_shared<lambertian>(checker)));

    return objects;
}


hittable_list two_perlin_spheres() {
    hittable_list objects;

    auto pertext = make_shared<noise_texture>(4);
    objects.add(make_shared<sphere>(point3(0,-1000,0), 1000, make_shared<lambertian>(pertext)));
    objects.add(make_shared<sphere>(point3(0,2,0), 2, make_shared<lambertian>(pertext)));

    return objects;
}


hittable_list earth() {
    auto earth_texture = make_shared<image_texture>("raytracing.github.io/images/earthmap.jpg");
    auto earth_surface = make_shared<lambertian>(earth_texture);
    auto globe = make_shared<sphere>(point3(0,0,0), 2, earth_surface);

    return hittable_list(globe);
}


hittable_list simple_light() {
    hittable_list objects;

    auto pertext = make_shared<noise_texture>(4);
    objects.add(make_shared<sphere>(point3(0,-1000,0), 1000, make_shared<lambertian>(pertext)));
    objects.add(make_shared<sphere>(point3(0,2,0), 2, make_shared<lambertian>(pertext)));

    auto difflight = make_shared<diffuse_light>(color(4,4,4));
    objects.add(make_shared<sphere>(point3(0,7,0), 2, difflight));
    objects.add(make_shared<xy_rect>(3, 5, 1, 3, -2, difflight));

    return objects;
}


hittable_list cornell_box() {
    hittable_list objects;

    auto red   = make_shared<lambertian>(color(.65, .05, .05));
    auto white = make_shared<lambertian>(color(.73, .73, .73));
    auto green = make_shared<lambertian>(color(.12, .45, .15));
    auto light = make_shared<diffuse_light>(color(15, 15, 15));

    objects.add(make_shared<yz_rect>(0, 555, 0, 555, 555, green));
    objects.add(make_shared<yz_rect>(0, 555, 0, 555, 0, red));
    objects.add(make_shared<xz_rect>(213, 343, 227, 332, 554, light));
    objects.add(make_shared<xz_rect>(0, 555, 0, 555, 555, white));
    objects.add(make_shared<xz_rect>(0, 555, 0, 555, 0, white));
    objects.add(make_shared<xy_rect>(0, 555, 0, 555, 555, white));

    shared_ptr<hittable> box1 = make_shared<box>(point3(0,0,0), point3(165,330,165), white);
    box1 = make_shared<rotate_y>(box1, 15);
    box1 = make_shared<translate>(box1, vec3(265,0,295));
    objects.add(box1);

    shared_ptr<hittable> box2 = make_shared<box>(point3(0,0,0), point3(165,165,165), white);
    box2 = make_shared<rotate_y>(box2, -18);
    box2 = make_shared<translate>(box2, vec3(130,0,65));
    objects.add(box2);

    return objects;
}


hittable_list cornell_smoke() {
    hittable_list objects;

    auto red   = make_shared<lambertian>(color(.65, .05, .05));
    auto white = make_shared<lambertian>(color(.73, .73, .73));
    auto green = make_shared<lambertian>(color(.12, .45, .15));
    auto light = make_shared<diffuse_light>(color(7, 7, 7));

    objects.add(make_shared<yz_rect>(0, 555, 0, 555, 555, green));
    objects.add(make_shared<yz_rect>(0, 555, 0, 555, 0, red));
    objects.add(make_shared<xz_rect>(113, 443, 127, 432, 554, light));
    objects.add(make_shared<xz_rect>(0, 555, 0, 555, 555, white));
    objects.add(make_shared<xz_rect>(0, 555, 0, 555, 0, white));
    objects.add(make_shared<xy_rect>(0, 555, 0, 555, 555, white));

    shared_ptr<hittable> box1 = make_shared<box>(point3(0,0,0), point3(165,330,165), white);
    box1 = make_shared<rotate_y>(box1, 15);
    box1 = make_shared<translate>(box1, vec3(265,0,295));

    shared_ptr<hittable> box2 = make_shared<box>(point3(0,0,0), point3(165,165,165), white);
    box2 = make_shared<rotate_y>(box2, -18);
    box2 = make_shared<translate>(box2, vec3(130,0,65));

    objects.add(make_shared<constant_medium>(box1, 0.01, color(0,0,0)));
    objects.add(make_shared<constant_medium>(box2, 0.01, color(1,1,1)));

    return objects;
}


hittable_list final_scene() {
    hittable_list boxes1;
    auto ground = make_shared<lambertian>(color(0.48, 0.83, 0.53));

    const int boxes_per_side = 20;
    for (int i = 0; i < boxes_per_side; i++) {
        for (int j = 0; j < boxes_per_side; j++) {
            auto w = 100.0;
            auto x0 = -1000.0 + i*w;
            auto z0 = -1000.0 + j*w;
            auto y0 = 0.0;
            auto x1 = x0 + w;
            auto y1 = random_double(1,101);
            auto z1 = z0 + w;

            boxes1.add(make_shared<box>(point3(x0,y0,z0), point3(x1,y1,z1), ground));
        }
    }

    hittable_list objects;

    objects.add(make_shared<bvh_node>(boxes1, 0, 1));

    auto light = make_shared<diffuse_light>(color(7, 7, 7));
    objects.add(make_shared<xz_rect>(123, 423, 147, 412, 554, light));

    auto center1 = point3(400, 400, 200);
    auto center2 = center1 + vec3(30,0,0);
    auto moving_sphere_material = make_shared<lambertian>(color(0.7, 0.3, 0.1));
    objects.add(make_shared<moving_sphere>(center1, center2, 0, 1, 50, moving_sphere_material));

    objects.add(make_shared<sphere>(point3(260, 150, 45), 50, make_shared<dielectric>(1.5)));
    objects.add(make_shared<sphere>(
        point3(0, 150, 145), 50, make_shared<metal>(color(0.8, 0.8, 0.9), 1.0)
    ));

    auto boundary = make_shared<sphere>(point3(360,150,145), 70, make_shared<dielectric>(1.5));
    objects.add(boundary);
    objects.add(make_shared<constant_medium>(boundary, 0.2, color(0.2, 0.4, 0.9)));
    boundary = make_shared<sphere>(point3(0,0,0), 5000, make_shared<dielectric>(1.5));
    objects.add(make_shared<constant_medium>(boundary, .0001, color(1,1,1)));

    auto emat = make_shared<lambertian>(make_shared<image_texture>("earthmap.jpg"));
    objects.add(make_shared<sphere>(point3(400,200,400), 100, emat));
    auto pertext = make_shared<noise_texture>(0.1);
    objects.add(make_shared<sphere>(point3(220,280,300), 80, make_shared<lambertian>(pertext)));

    hittable_list boxes2;
    auto white = make_shared<lambertian>(color(.73, .73, .73));
    int ns = 1000;
    for (int j = 0; j < ns; j++) {
        boxes2.add(make_shared<sphere>(point3::random(0,165), 10, white));
    }

    objects.add(make_shared<translate>(
        make_shared<rotate_y>(
            make_shared<bvh_node>(boxes2, 0.0, 1.0), 15),
            vec3(-100,270,395)
        )
    );

    return objects;
}

// std::clamp 
double clamp(double r, double a, double b){
    if (r < a) return a;
    if (r > b) return b;
    return r;
}

int main() {

    // Image

    auto aspect_ratio = 16.0 / 9.0;
    int image_width = 100;
    int samples_per_pixel = 10;
    int max_depth = 50;

    // World

    hittable_list world;

    point3 lookfrom;
    point3 lookat;
    auto vfov = 40.0;
    auto aperture = 0.0;
    color background(0,0,0);

    switch (2) {
        case 1:
            world = random_scene();
            background = color(0.70, 0.80, 1.00);
            lookfrom = point3(13,2,3);
            lookat = point3(0,0,0);
            vfov = 20.0;
            aperture = 0.1;
            break;

        case 2:
            world = two_spheres();
            background = color(0.70, 0.80, 1.00);
            lookfrom = point3(13,2,3);
            lookat = point3(0,0,0);
            vfov = 20.0;
            break;

        case 3:
            world = two_perlin_spheres();
            background = color(0.70, 0.80, 1.00);
            lookfrom = point3(13,2,3);
            lookat = point3(0,0,0);
            vfov = 20.0;
            break;

        case 4:
            world = earth();
            background = color(0.70, 0.80, 1.00);
            lookfrom = point3(0,0,12);
            lookat = point3(0,0,0);
            vfov = 20.0;
            break;

        case 5:
            world = simple_light();
            samples_per_pixel = 400;
            lookfrom = point3(26,3,6);
            lookat = point3(0,2,0);
            vfov = 20.0;
            break;

        default:
        case 6:
            world = cornell_box();
            aspect_ratio = 1.0;
            image_width = 600;
            samples_per_pixel = 200;
            lookfrom = point3(278, 278, -800);
            lookat = point3(278, 278, 0);
            vfov = 40.0;
            break;

        case 7:
            world = cornell_smoke();
            aspect_ratio = 1.0;
            image_width = 600;
            samples_per_pixel = 200;
            lookfrom = point3(278, 278, -800);
            lookat = point3(278, 278, 0);
            vfov = 40.0;
            break;

        case 8:
            world = final_scene();
            aspect_ratio = 1.0;
            image_width = 800;
            samples_per_pixel = 10000;
            lookfrom = point3(478, 278, -600);
            lookat = point3(278, 278, 0);
            vfov = 40.0;
            break;
    }

    // Camera

    const vec3 vup(0,1,0);
    const auto dist_to_focus = 10.0;
    const int image_height = static_cast<int>(image_width / aspect_ratio);

    camera cam(lookfrom, lookat, vup, vfov, aspect_ratio, aperture, dist_to_focus, 0.0, 1.0);

    // Render
    unsigned char data[image_width * image_height * CHANNEL_NUM];
    std::cout << "P3\n" << image_width << ' ' << image_height << "\n255\n";

    for (int j = image_height-1; j >= 0; --j) {
        std::cerr << "\rScanlines remaining: " << j << ' ' << std::flush;
        for (int i = 0; i < image_width; ++i) {
            color pixel_color(0,0,0);
            for (int s = 0; s < samples_per_pixel; ++s) {
                auto u = (i + random_double()) / (image_width-1);
                auto v = (j + random_double()) / (image_height-1);
                ray r = cam.get_ray(u, v);
                pixel_color += ray_color(r, background, world, max_depth);
            }
            //write_color(std::cout, pixel_color, samples_per_pixel);
            pixel_color = pixel_color / samples_per_pixel;

            auto r = pixel_color.x();
            auto g = pixel_color.y();
            auto b = pixel_color.z();

            int ir = static_cast<int>(256 * clamp(r, 0.0, 0.999));
            int ig = static_cast<int>(256 * clamp(g, 0.0, 0.999));
            int ib = static_cast<int>(256 * clamp(b, 0.0, 0.999));

            data[index++] = ir;
            data[index++] = ig;
            data[index++] = ib;
        }
    }
    stbi_write_png("imageRTnextWeek.png", image_width, image_height, CHANNEL_NUM, data, image_width*CHANNEL_NUM);
    std::cerr << "\nDone.\n";
}


Overwriting main.cc


### Run

In [ ]:
# compile and run 
! g++ -Iraytracing.github.io/src/common -std=c++17 main.cc -o RTtheNextWeek
! ./RTtheNextWeek

# Ray Tracing in a Weekend (Colab)

**Ray Trace in One Weekend in Google Colab**

In [34]:
!mkdir includeRT 

### rtw_stb_image.h

In [35]:
%%file includeRT/rtw_stb_image.h

#ifndef RTWEEKEND_STB_IMAGE_H
#define RTWEEKEND_STB_IMAGE_H


// Disable pedantic warnings for this external library.
#ifdef _MSC_VER
    // Microsoft Visual C++ Compiler
    #pragma warning (push, 0)
#endif



#define STB_IMAGE_IMPLEMENTATION
#include "raytracing.github.io/src/common/external/stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "raytracing.github.io/src/common/external/stb_image_write.h"

// Restore warning levels.
#ifdef _MSC_VER
    // Microsoft Visual C++ Compiler
    #pragma warning (pop)
#endif

#endif

Writing RTinclude/rtw_stb_image.h


https://stackoverflow.com/questions/56039401/stb-image-write-issue

### vec3.h

In [36]:
%%file includeRT/vec3.h

#ifndef VEC3_H
#define VEC3_H

#include <cmath>
#include <iostream>
#include <cstdlib>

using std::sqrt;
using std::fabs;

class vec3 {
    public:
        vec3() : e{0,0,0} {}
        vec3(double e0, double e1, double e2) : e{e0,e1,e2} {}

        double x() const { return e[0]; }
        double y() const { return e[1]; }
        double z() const { return e[2]; }

        vec3 operator-() const { return vec3(-e[0], -e[1], -e[2]); }
        double operator[](int i) const { return e[i]; }
        double& operator[](int i) { return e[i]; }

        vec3& operator+=(const vec3 &v) {
            e[0] += v.e[0];
            e[1] += v.e[1];
            e[2] += v.e[2];
            return *this;
        }

        vec3& operator*=(const double t) {
            e[0] *= t;
            e[1] *= t;
            e[2] *= t;
            return *this;
        }

        vec3& operator/=(const double t){
            return *this *= 1/t;
        }
        double length() const {
            return sqrt(length_squared());
        }
        double length_squared() const {
            return e[0]*e[0] + e[1]*e[1] + e[2]*e[2];
        }

        

        bool near_zero() const {
            const auto s = 1e-8;
            return (fabs(e[0]) < s) && (fabs(e[1]) < s) && (fabs(e[2]) < s);
        }
    public:
        double e[3];

};

// vec3 Utility Functions 

inline std::ostream& operator<<(std::ostream &out, const vec3 &v){
    return out << v.e[0] << ' ' << v.e[1] << ' ' << v.e[2];
}
inline vec3 operator+(const vec3 &u, const vec3 &v){
    return vec3(u.e[0] + v.e[0], u.e[1] + v.e[1], u.e[2] + v.e[2]);
}
inline vec3 operator-(const vec3 &u, const vec3 &v){
    return vec3(u.e[0] - v.e[0], u.e[1] - v.e[1], u.e[2] - v.e[2]);
}
inline vec3 operator*(const vec3 &u, const vec3 &v){
    return vec3(u.e[0] * v.e[0], u.e[1] * v.e[1], u.e[2] * v.e[2]);
}
inline vec3 operator*(double t, const vec3 &v){
    return vec3(t * v.e[0], t * v.e[1], t * v.e[2]);
}
inline vec3 operator*(const vec3 &v, double t){
    return t * v;
}
inline vec3 operator/(vec3 v, double t){
    return (1/t) * v;
}
inline double dot(const vec3 &u, const vec3 &v){
    return u.e[0] * v.e[0] + u.e[1] * v.e[1] + u.e[2] * v.e[2];
}
inline vec3 cross(const vec3 &u, const vec3 &v){
    return vec3(u.e[1]*v.e[2] - u.e[2]*v.e[1],
                u.e[2]*v.e[0] - u.e[0]*v.e[2],    // double check 
                u.e[0]*v.e[1] - u.e[1]*v.e[0]);
}
inline vec3 unit_vector(vec3 v){
    return v / v.length();
}

inline double random_double() {
          // Returns a random real in [0,1).
          return rand() / (RAND_MAX + 1.0);
        }

inline double random_double(double min, double max) {
            // Returns a random real in [min,max).
            return min + (max-min)*random_double();
        }        

inline static vec3 random(double min, double max) {
    return vec3(random_double(min, max), random_double(min, max), random_double(min, max));
}

inline vec3 random_in_unit_sphere(){
    while(true){
        auto p = random(-1,1);
        if (p.length_squared() >= 1) continue;
        return p;
    }
}
vec3 random_unit_vector(){
    return unit_vector(random_in_unit_sphere());
}
vec3 reflect(const vec3& v, const vec3& n){
    return v - 2*dot(v,n)*n; // v + 2b
}
vec3 random_in_unit_disk(){
    while(true){
        auto p = vec3(random_double(-1,1), random_double(-1,1), 0);
        if (p.length_squared() >= 1) continue;
        return p;
    }
}
// Type aliases for vec3
using point3 = vec3;    // 3D point
using color = vec3;    // RGB color

#endif

Writing RTinclude/vec3.h


### color.h

R G B Color\
0.0 0.0 0.0 Black\
1.0 0.0 0.0 Red\
0.0 1.0 0.0 Green\
0.0 0.0 1.0 Blue\
1.0 1.0 0.0 Yellow\
1.0 0.0 1.0 Magenta\
0.0 1.0 1.0 Cyan\
1.0 1.0 1.0 White

In [37]:
%%file includeRT/color.h 

#ifndef COLOR_H
#define COLOR_H

#include "vec3.h"
#include <iostream>

void write_color(color &pixel_out, color pixel_color){
    
    pixel_out = pixel_color;
}

#endif

Writing RTinclude/color.h


ray.h

In [38]:
%%file includeRT/ray.h

#ifndef RAY_H
#define RAY_H

#include "vec3.h"
// ray::at(t) = orig + t*dir 
class ray {
    public:
        ray() {}
        ray(const point3& origin, const vec3& direction, double time = 0.0)
          : orig(origin), dir(direction), tm(time) 
        {}

        point3 origin() const { return orig; }
        vec3 direction() const { return dir; }
        double time() const { return tm; }  
        point3 at(double t) const {
            return orig + t*dir;
        }
    
    public:
        point3 orig;
        vec3 dir;
        double tm;
};

#endif

Writing RTinclude/ray.h


### rtweekend.h

In [39]:
%%file includeRT/rtweekend.h

#ifndef RTWEEKEND_H
#define RTWEEKEND_H

#include <cmath>
#include <limits>
#include <memory>

// Usings
using std::shared_ptr;
using std::make_shared;
using std::sqrt;


// Constants
const double infinity = std::numeric_limits<double>::infinity();
const double pi = 3.1415926535897932385;

// Utility Functions

inline double degrees_to_radians(double degrees){
    return pi*degrees / 180.0;
}

// Common Headers

#include "ray.h"
#include "vec3.h"

#endif

Writing RTinclude/rtweekend.h


### hittable.h

In [40]:
%%file includeRT/hittable.h

#ifndef HITTABLE_H
#define HITTABLE_H

#include "rtweekend.h"

class material;

struct hit_record{
    point3 p;
    vec3 normal;
    double t; // root
    shared_ptr<material> mat_ptr;
};

class hittable {
    public:
      virtual bool hit(const ray& r, double t_tmin, double t_max, hit_record& rec) const = 0;
};

#endif

Writing RTinclude/hittable.h


### sphere.h

In [41]:
%%file includeRT/sphere.h

#ifndef SPHERE_H
#define SPHERE_H

#include "hittable.h"
#include "vec3.h"

class sphere : public hittable {
    public:
      sphere(){}
      sphere(point3 cen, double r, shared_ptr<material> m) : center(cen), radius(r), mat_ptr(m) {};

      virtual bool hit(const ray& r, double t_min, double t_max, hit_record& rec) const override;
    public:
      point3 center;
      double radius;
      shared_ptr<material> mat_ptr;
};

bool sphere::hit(const ray& r, double t_min, double t_max, hit_record& rec) const {
  // t = (-b+/- sqrt(b*b-4*a*c)) / (2*a)
  // t = (-h+/- sqrt(h*h-a*c)) / (a)
  // b = 2*h
  vec3 oc = r.origin() - center; // A - C
  auto half_b = dot(r.direction(), oc); // h = b/2 = B dot (A-C)
  auto a = r.direction().length_squared(); // a = B dot B 
  auto c = dot(oc, oc) - radius*radius; // c = (A-C) dot (A-C) - r^2
  auto determinant = half_b*half_b - a*c;
  if (determinant < 0)
    return false;
  else {
    auto sqrtd = sqrt(determinant); 
    // Find nearest root that lies in the acceptable range 
    auto root = (-half_b-sqrtd)/a;
    // t_min < root < t_max
    if (root < t_min || t_max < root){
        root = (-half_b+sqrtd)/a;
        if (root < t_min || t_max < root)
          return false;
    }
    rec.t = root;
    rec.p = r.at(rec.t);
    rec.normal = (rec.p - center)/radius;  // normal = (P - C) / |P - C|
    rec.mat_ptr = mat_ptr;
    return true;
  }
}
#endif 

Writing RTinclude/sphere.h


### hittable_list.h

In [42]:
%%file includeRT/hittable_list.h

#ifndef HITTABLE_LIST_H
#define HITTABLE_LIST_H

#include "hittable.h"
#include <memory>
#include <vector>

using std::shared_ptr;
using std::make_shared;

class hittable_list : public hittable {
    public:
      hittable_list() {}
      
      hittable_list(shared_ptr<hittable> object) { add(object); }

      void clear() { objects.clear();}
      void add(shared_ptr<hittable> object) { objects.push_back(object); }
      virtual bool hit(const ray& r, double t_min, double t_max, hit_record& rec) const override;

    public:
      std::vector<shared_ptr<hittable>> objects;
};

bool hittable_list::hit(const ray& r, double t_min, double t_max, hit_record& rec) const {
    
    hit_record temp_rec;
    bool hit_anything = false;
    auto closest_so_far = t_max;

    for (const auto& object : objects) {
        if (object->hit(r, t_min, closest_so_far, temp_rec)) {
            hit_anything = true;
            closest_so_far = temp_rec.t;
            rec = temp_rec;
        }
    }

    return hit_anything;
}

#endif

Writing RTinclude/hittable_list.h


### material.h

In [43]:
%%file includeRT/material.h 

#ifndef MATERIAL_H
#define MATERIAL_H 

#include "rtweekend.h"

struct hit_record;

class material{
    public:
      virtual bool scatter(const ray& r_in, const hit_record& rec, color& attenuation, ray& scattered) const = 0;
};

class lambertian : public material {
    public:
      lambertian(const color& a) : albedo(a) {}

      virtual bool scatter(const ray& r_in, const hit_record& rec, color& attenuation, ray& scattered) const override {
          auto scattered_direction = rec.normal + random_unit_vector();
          // Catch degenerate scatter direction 
          if (scattered_direction.near_zero())
            scattered_direction = rec.normal;
          scattered = ray(rec.p, scattered_direction);  // ray(origin, direction)
          attenuation = albedo;
          return true;
      }
    public:
      color albedo;
};

class metal : public material {
    public:
      metal(const color& a) : albedo(a) {}

      virtual bool scatter(const ray& r_in, const hit_record& rec, color& attenuation, ray& scattered) const override
       {
          //auto scattered_direction = rec.normal + random_unit_vector();
          // Catch degenerate scatter direction 
          vec3 reflected = reflect(unit_vector(r_in.direction()), rec.normal);
          scattered = ray(rec.p, reflected);  // ray(origin, direction)
          attenuation = albedo;
          return true;
      }
    public:
      color albedo;
};

#endif

Writing RTinclude/material.h


### camera.h

In [44]:
%%file includeRT/camera.h 

#ifndef CAMERA_H
#define CAMERA_H

#include "rtweekend.h"

class camera {
    public:
      camera() : camera(point3(0,0,0), point3(0,0,-1), point3(0,1,0), 40.0, 1.0, 1.0, 1.0, 0.0, 0.0) {}
      camera(
          point3 lookfrom,
          point3 lookat,
          point3 vup,
          double vfov, // vertical field-of-view in degrees 
          double aspect_ratio,
          double aperture,
          double focus_dist,
          double _time0 = 0,
          double _time1 = 0
      ){
          //auto aspect_ratio = 16.0/9.0; 
          //auto FOV = 90.0;
          auto theta = degrees_to_radians(vfov);
          auto h = tan(theta/2);
          auto viewport_height = 2.0*h;
          auto viewport_width = aspect_ratio * viewport_height;
          //auto focal_length = 1.0;

          origin = lookfrom;
          w = unit_vector(lookfrom-lookat);
          u = unit_vector(cross(vup, w));
          v = cross(w,u);
          horizontal = focus_dist*viewport_width*u;
          vertical = focus_dist*viewport_height*v;
          lower_left_corner = origin - horizontal/2 - vertical/2 - focus_dist*w;
        
          lens_radius = aperture / 2;
          time0 = _time0;
          time1 = _time1;
      }

      ray get_ray(double s, double t) const {
          vec3 rd = lens_radius*random_in_unit_disk();
          vec3 offset = u*rd.x() + v*rd.y();
          return ray(origin + offset, 
                     lower_left_corner + s*horizontal + t*vertical - origin - offset,
                     random_double(time0, time1));    // ray(orig, dir) 
      }
    private:
        point3 origin;
        point3 lower_left_corner;
        vec3 horizontal;
        vec3 vertical;
        vec3 u,v,w;
        double lens_radius;
        double time0, time1; // shutter open / close times 
};

#endif

Writing RTinclude/camera.h


## main.cc

3 components for Ray Tracing : World, Object and Camera
1.   Create Camera
2.   Create World
3.   Create multiple objects in the world
4.   Specify materials (dielectric, metal etc) 
5.   Modify camera




In [32]:
%%file main.cc

#include "rtw_stb_image.h"

#include "rtweekend.h"

#include "color.h"
#include "hittable_list.h"
#include "sphere.h"
#include "material.h"
#include "camera.h"

#include <iostream>
//#include <algorithm>  // std::clamp

#define CHANNEL_NUM 3

// std::clamp 
double clamp(double r){
    if (r < 0.0) return 0.0;
    if (r > 0.999) return 0.999;

    return r;
}
// render surface normals on a sphere
color ray_color(const ray& r, const hittable_list& world, int depth){
  // blendedValue = (1-t)*startValue + t*endValue
  // depth = ray bounce limit
  if (depth <=0)
    return color(0,0,0);

  hit_record rec;
  if (world.hit(r, 0.001, infinity, rec)){
      ray scattered;
      color attenuation;
      if (rec.mat_ptr -> scatter(r, rec, attenuation, scattered))
        return attenuation*ray_color(scattered, world, depth-1);

      return color(0,0,0);
  }

  vec3 unit_direction = unit_vector(r.direction());
  auto t = 0.5*(unit_direction.y() + 1.0);
  return (1.0-t)*color(1.0,1.0,1.0) + t*color(0.5,0.7,1.0);    
}

hittable_list random_scene(){
    hittable_list world;

    auto material_ground = make_shared<lambertian>(color(0.8,0.8,0));
    auto material_center = make_shared<lambertian>(color(0.7,0.3,0.3));
    auto material_left = make_shared<lambertian>(color(0,0,1.0));  // blue
    auto material_right = make_shared<metal>(color(1,0,0));  // red 
    
    auto wide_angle_R = 1;  // wide angle camera
    
    world.add(make_shared<sphere>(point3(0,-100.5,-1), 100, material_ground));  // ground
    world.add(make_shared<sphere>(point3(0,0,-1), 0.5, material_center)); // center
    world.add(make_shared<sphere>(point3(-wide_angle_R,0,-1), 0.5, material_left)); // left 
    world.add(make_shared<sphere>(point3(wide_angle_R,0,-1), 0.5, material_right));  // right   
    
    return world;
}

int main(){
    
    // Image
    const int image_width = 400;
    const auto aspect_ratio = 16.0 / 9.0;
    const int image_height = static_cast<int> (image_width / aspect_ratio);
    const int max_depth = 50;
    int samples_per_pixel = 100;
    // World
    auto world = random_scene();

    // Camera 
    point3 lookfrom(-2,2,3);
    point3 lookat(0,0,0);
    point3 vup(0,1,0);
    auto dist_to_focus = (lookfrom-lookat).length();
    auto aperture = 0.1;
    camera cam(lookfrom, lookat, vup, 40.0, aspect_ratio, aperture, dist_to_focus, 0.0, 1.0);  
          // lookfrom, lookat, vup, vertical FOV, aspect ratio, aperture, focus distance, shutter open time, shutter close time
          
    // Render 
    unsigned char data2[image_width * image_height * CHANNEL_NUM];
    std::cout << "P3\n" << image_width << ' ' << image_height << "\n255\n";

    int index = 0; 
    for (int j = image_height-1; j >=0 ; --j){
        std::cerr << "\rScanlines remaining " << j << ' ' << std::flush;
        for (int i = 0; i < image_width; i++){
            
            //ray R(origin, lower_left_corner + u*horizontal + v*vertical - origin);
            
            color pixel_color(0,0,0);
            
            
            for (int k =0; k < samples_per_pixel; k++){
                auto u = (double(i) + random_double()) / (image_width-1); 
                auto v = (double(j) + random_double()) / (image_height-1);
                ray R = cam.get_ray(u,v);
                pixel_color += ray_color(R, world, max_depth);  // traced ray in the created world
            } 
            
            pixel_color = pixel_color / samples_per_pixel;

            auto r = pixel_color.x();
            auto g = pixel_color.y();
            auto b = pixel_color.z();
            
            //auto r = double(i) / (image_width-1);
            //auto g = double(j) / (image_height-1);
            //auto b = 0.25;
            //color pixel_in(r,g,b);
            //color pixel_out;
            int ir = static_cast<int>(256 * clamp(r));
            int ig = static_cast<int>(256 * clamp(g));
            int ib = static_cast<int>(256 * clamp(b));
            //write_color(pixel_out, pixel_in);
            // RGB --> BRG 
            //data2[j][i].R = ir;
            //data2[j][i].G = ig;
            //data2[j][i].B = ib;
            data2[index++] = ir;
            data2[index++] = ig;
            data2[index++] = ib;
            //std::cout << ir << ' ' << ig << ' ' << ib << ' ';
            //std::cout << data1[j][i] << '\n';
        }
        //std::cout << "\n" << "new line" << "\n";
    }

    stbi_write_png("image1.png", image_width, image_height, CHANNEL_NUM, data2, image_width*CHANNEL_NUM);
    std::cerr <<"\nDone\n";
    return 0;
}


Overwriting main.cc


## Run

In [45]:
# compile and run 
! g++ -IincludeRT -std=c++17 main.cc -o introRayTrace
! ./introRayTrace

P3
400 225
255
Scanlines remaining 0 
Done


# PBRT V3

Resources



*   https://hackmd.io/@nycu-cas/H1HeJ2A7Y
*   https://graphics.stanford.edu/wikis/cs348b-07/Assignment3
*   https://graphics.stanford.edu/courses/cs348b-04/homework/hw3.html



In [2]:
import os
if os.path.exists('pbrt-v3'):
  print('Use local pbrt-v3 repo.')
else:
  print('Checking out pbrt-v3 repo ...')
  !git clone --recursive https://github.com/mmp/pbrt-v3/

Checking out pbrt-v3 repo ...
Cloning into 'pbrt-v3'...
remote: Enumerating objects: 6264, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6264 (delta 5), reused 5 (delta 5), pack-reused 6256
Receiving objects: 100% (6264/6264), 5.59 MiB | 11.74 MiB/s, done.
Resolving deltas: 100% (4820/4820), done.
Submodule 'src/ext/glog' (https://github.com/google/glog.git) registered for path 'src/ext/glog'
Submodule 'src/ext/openexr' (https://github.com/wjakob/openexr) registered for path 'src/ext/openexr'
Submodule 'src/ext/ptex' (https://github.com/wdas/ptex.git) registered for path 'src/ext/ptex'
Submodule 'src/ext/zlib' (https://github.com/mitsuba-renderer/zlib) registered for path 'src/ext/zlib'
Cloning into '/content/pbrt-v3/src/ext/glog'...
remote: Enumerating objects: 3866, done.        
remote: Counting objects: 100% (128/128), done.        
remote: Compressing objects: 100% (69/69), done.        
remote: Total 3866 (delta 68)

In [3]:
!mkdir pbrt-v3/build

In [4]:
%cd pbrt-v3/build/

/content/pbrt-v3/build


In [5]:
!cmake ..

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Setting build type to 'Release' as none was specified.
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Looking for include file alloca.h
-- Looking for include file alloca.h -

In [6]:
!ls

CMakeCache.txt	cmake_install.cmake  CPackSourceConfig.cmake  Makefile
CMakeFiles	CPackConfig.cmake    CTestTestfile.cmake      src


In [6]:
!make 

[  0%] Building CXX object CMakeFiles/pbrt.dir/src/core/api.cpp.o
[  1%] Building CXX object CMakeFiles/pbrt.dir/src/core/bssrdf.cpp.o
[  1%] Building CXX object CMakeFiles/pbrt.dir/src/core/camera.cpp.o
[  1%] Building CXX object CMakeFiles/pbrt.dir/src/core/efloat.cpp.o
[  2%] Building CXX object CMakeFiles/pbrt.dir/src/core/error.cpp.o
[  2%] Building CXX object CMakeFiles/pbrt.dir/src/core/fileutil.cpp.o
[  2%] Building CXX object CMakeFiles/pbrt.dir/src/core/film.cpp.o
[  3%] Building CXX object CMakeFiles/pbrt.dir/src/core/filter.cpp.o
[  3%] Building CXX object CMakeFiles/pbrt.dir/src/core/floatfile.cpp.o
[  3%] Building CXX object CMakeFiles/pbrt.dir/src/core/geometry.cpp.o
[  4%] Building CXX object CMakeFiles/pbrt.dir/src/core/imageio.cpp.o
In file included from /content/pbrt-v3/src/core/imageio.cpp:36:
/content/pbrt-v3/src/ext/targa.h:48: warning: "htole16" redefined
   48 | #define htole16(x) (x)
      | 
In file included from /usr/include/x86_64-linux-gnu/sys/types.h:176,


In [7]:
%cd ..
%cd .. 

/content/pbrt-v3
/content


In [9]:
%ls

gdrive/  pbrt-v3/  sample_data/


In [ ]:
! ./pbrt-v3/build/pbrt_test

download EXR file convert to PNG file https://convertio.co/exr-png/ 

In [ ]:
! ./pbrt-v3/build/pbrt /pbrt-v3/scenes/killeroo-simple.pbrt

https://www.pbrt.org/users-guide

https://www.pbrt.org/fileformat-v3#overview



## Build an example PBRT scene

In [15]:
%%file pbrt-v3/scenes/cornell-box.pbrt 

LookAt 278 273 -800  278 273 0   0 1 0
Camera "perspective" "float fov" [37.5]
PixelFilter "mitchell" "float xwidth" [3] "float ywidth" [3]
# Accelerator "kdtree"

Sampler "halton" "integer pixelsamples" 64

#Sampler "bestcandidate" "integer pixelsamples" [64]
Film "image"
   "string filename" ["cornell.png"]
   "integer xresolution" [480]
   "integer yresolution" [480]

Integrator "path"

# Renderer "metropolis" "integer samplesperpixel" [128] "bool dodirectseparately" ["false"]

WorldBegin

# light source

AttributeBegin
   AreaLightSource "diffuse" "spectrum L"    [400.0 0.0 500.0 8.0 600.0 15.6 700.0 18.4]
   Material "matte"
   "spectrum Kd" [400.0 0.0 500.0 8.0 600.0 15.6 700.0 18.4]

   Shape "trianglemesh"
   "integer indices" [ 0 1 2  2 3 0 ]
   "point P" [
      343.0 548.7 227.0
      343.0 548.7 332.0
      213.0 548.7 332.0
      213.0 548.7 227.0 ]
AttributeEnd

#red wall

AttributeBegin
   Material "matte"
   "spectrum Kd" [
      400 0.040 404 0.046 408 0.048 412 0.053 416 0.049 420 0.050
      424 0.053 428 0.055 432 0.057 436 0.056 440 0.059 444 0.057
      448 0.061 452 0.061 456 0.060 460 0.062 464 0.062 468 0.062
      472 0.061 476 0.062 480 0.060 484 0.059 488 0.057 492 0.058
      496 0.058 500 0.058 504 0.056 508 0.055 512 0.056 516 0.059
      520 0.057 524 0.055 528 0.059 532 0.059 536 0.058 540 0.059
      544 0.061 548 0.061 552 0.063 556 0.063 560 0.067 564 0.068
      568 0.072 572 0.080 576 0.090 580 0.099 584 0.124 588 0.154
      592 0.192 596 0.255 600 0.287 604 0.349 608 0.402 612 0.443
      616 0.487 620 0.513 624 0.558 628 0.584 632 0.620 636 0.606
      640 0.609 644 0.651 648 0.612 652 0.610 656 0.650 660 0.638
      664 0.627 668 0.620 672 0.630 676 0.628 680 0.642 684 0.639
      688 0.657 692 0.639 696 0.635 700 0.642]
   
   Shape "trianglemesh"
   "integer indices" [ 0 1 2  2 3 0 ]
   "point P" [
      552.8   0.0   0.0
      549.6   0.0 559.2
      556.0 548.8 559.2
      556.0 548.8   0.0 ]
AttributeEnd

# green wall

AttributeBegin
   Material "matte"
   "spectrum Kd" [
      400 0.092 404 0.096 408 0.098 412 0.097 416 0.098 420 0.095
      424 0.095 428 0.097 432 0.095 436 0.094 440 0.097 444 0.098
      448 0.096 452 0.101 456 0.103 460 0.104 464 0.107 468 0.109
      472 0.112 476 0.115 480 0.125 484 0.140 488 0.160 492 0.187
      496 0.229 500 0.285 504 0.343 508 0.390 512 0.435 516 0.464
      520 0.472 524 0.476 528 0.481 532 0.462 536 0.447 540 0.441
      544 0.426 548 0.406 552 0.373 556 0.347 560 0.337 564 0.314
      568 0.285 572 0.277 576 0.266 580 0.250 584 0.230 588 0.207
      592 0.186 596 0.171 600 0.160 604 0.148 608 0.141 612 0.136
      616 0.130 620 0.126 624 0.123 628 0.121 632 0.122 636 0.119
      640 0.114 644 0.115 648 0.117 652 0.117 656 0.118 660 0.120
      664 0.122 668 0.128 672 0.132 676 0.139 680 0.144 684 0.146
      688 0.150 692 0.152 696 0.157 700 0.159]

   Shape "trianglemesh"
   "integer indices" [ 0 1 2  2 3 0 ]
   "point P" [
      0.0   0.0 559.2
      0.0   0.0   0.0
      0.0 548.8   0.0
      0.0 548.8 559.2 ]
AttributeEnd

# everything else is white



Material "matte"

"spectrum Kd" [
      400 0.343 404 0.445 408 0.551 412 0.624 416 0.665 420 0.687
      424 0.708 428 0.723 432 0.715 436 0.710 440 0.745 444 0.758
      448 0.739 452 0.767 456 0.777 460 0.765 464 0.751 468 0.745
      472 0.748 476 0.729 480 0.745 484 0.757 488 0.753 492 0.750
      496 0.746 500 0.747 504 0.735 508 0.732 512 0.739 516 0.734
      520 0.725 524 0.721 528 0.733 532 0.725 536 0.732 540 0.743
      544 0.744 548 0.748 552 0.728 556 0.716 560 0.733 564 0.726
      568 0.713 572 0.740 576 0.754 580 0.764 584 0.752 588 0.736
      592 0.734 596 0.741 600 0.740 604 0.732 608 0.745 612 0.755
      616 0.751 620 0.744 624 0.731 628 0.733 632 0.744 636 0.731
      640 0.712 644 0.708 648 0.729 652 0.730 656 0.727 660 0.707
      664 0.703 668 0.729 672 0.750 676 0.760 680 0.751 684 0.739
      688 0.724 692 0.730 696 0.740 700 0.737 ]

#
# floor
#

AttributeBegin
   Shape "trianglemesh"
   "integer indices" [ 0 1 2  2 3 0 ]
   "point P" [
      552.8 0.0   0.0
      0.0 0.0   0.0
      0.0 0.0 559.2
    549.6 0.0 559.2 ]
AttributeEnd


#
# ceiling
#

AttributeBegin
   Shape "trianglemesh"
   "integer indices" [ 0 1 2  2 3 0 ]
   "point P" [
    556.0 548.8 0.0
    556.0 548.8 559.2
      0.0 548.8 559.2
      0.0 548.8   0.0 ]
AttributeEnd


#
# back wall
#

AttributeBegin
   Shape "trianglemesh"
   "integer indices" [ 0 1 2  2 3 0 ]
   "point P" [
    549.6   0.0 559.2
      0.0   0.0 559.2
      0.0 548.8 559.2
    556.0 548.8 559.2]
AttributeEnd


#
# short block
#

# material { glass ior 1.5 r { constant spd { 1 0.95 } } }

AttributeBegin
   Shape "trianglemesh"
   "integer indices" [
      3 2 0 2 1 0
      7 6 4 6 5 4
      11 10 8 10 9 8
      15 14 12 14 13 12
      19 18 16 18 17 16
   ]
   "point P" [
    130.0 165.0  65.0
     82.0 165.0 225.0
    240.0 165.0 272.0
    290.0 165.0 114.0
    290.0   0.0 114.0
    290.0 165.0 114.0
    240.0 165.0 272.0
    240.0   0.0 272.0
    130.0   0.0  65.0
    130.0 165.0  65.0
    290.0 165.0 114.0
    290.0   0.0 114.0
     82.0   0.0 225.0
     82.0 165.0 225.0
    130.0 165.0  65.0
    130.0   0.0  65.0
    240.0   0.0 272.0
    240.0 165.0 272.0
     82.0 165.0 225.0
     82.0   0.0 225.0 ]
AttributeEnd


#
# tall block
#

AttributeBegin
   Shape "trianglemesh"
   "integer indices" [
      3 2 0 2 1 0
      7 6 4 6 5 4
      11 10 8 10 9 8
      15 14 12 14 13 12
      19 18 16 18 17 16
   ]
   "point P" [
    423.0 330.0 247.0
    265.0 330.0 296.0
    314.0 330.0 456.0
    472.0 330.0 406.0
    423.0   0.0 247.0
    423.0 330.0 247.0
    472.0 330.0 406.0
    472.0   0.0 406.0
    472.0   0.0 406.0
    472.0 330.0 406.0
    314.0 330.0 456.0
    314.0   0.0 456.0
    314.0   0.0 456.0
    314.0 330.0 456.0
    265.0 330.0 296.0
    265.0   0.0 296.0
    265.0   0.0 296.0
    265.0 330.0 296.0
    423.0 330.0 247.0
    423.0   0.0 247.0]
AttributeEnd

WorldEnd

Overwriting pbrt-v3/scenes/cornell-box.pbrt


http://vda.univie.ac.at/Teaching/ImageSynthesis/17w/Lab2/lab2.html

In [16]:
! ./pbrt-v3/build/pbrt pbrt-v3/scenes/cornell-box.pbrt

pbrt version 3 (built Jan 31 2023 at 03:36:31) [Detected 2 cores]
Copyright (c)1998-2018 Matt Pharr, Greg Humphreys, and Wenzel Jakob.
The source code to pbrt (but *not* the book contents) is covered by the BSD License.
See the file LICENSE.txt for the conditions of the license.
Rendering: [++]  (92.5s)       
Statistics:
  BVH
    Interior nodes                                                     19
    Leaf nodes                                                         20
    Primitives per leaf node                            32 /           20 (1.60x)
  Integrator
    Camera rays traced                                           15116544
    Path length                                                     2.918 avg [range 0 - 5]
    Zero-radiance paths                           18984806 /     49404998 (38.43%)
  Intersections
    Regular ray intersection tests                               59392786
    Shadow ray intersection tests                                34714228
    Ray-triang

In [41]:
%%file pbrt-v3/scenes/example.pbrt

LookAt 3 4 1.5  # eye
       .5 .5 0  # look at point
       0 0 1    # up vector
Camera "perspective" "float fov" 40 "lensradius" 0.5 "focaldistance" 5.0

Sampler "halton" "integer pixelsamples" 128
Integrator "path"
Film "image" "string filename" "simple.png"
     "integer xresolution" [400] "integer yresolution" [400]

WorldBegin

# uniform blue-ish illumination from all directions
LightSource "infinite" "rgb L" [.4 .45 .5]

# approximate the sun
LightSource "distant"  "point from" [ -30 40  100 ]
   "blackbody L" [3000 1.5]

AttributeBegin
  Material "glass"
  Shape "sphere" "float radius" 1
AttributeEnd

AttributeBegin
  Texture "checks" "spectrum" "checkerboard"
          "float uscale" [8] "float vscale" [8]
          "rgb tex1" [.1 .1 .1] "rgb tex2" [.8 .8 .8]
  Material "matte" "texture Kd" "checks"
  Translate 0 0 -1
  Shape "trianglemesh"
      "integer indices" [0 1 2 0 2 3]
      "point P" [ -20 -20 0   20 -20 0   20 20 0   -20 20 0 ]
      "float st" [ 0 0   1 0    1 1   0 1 ]
AttributeEnd

WorldEnd

Overwriting scenes/example.pbrt


In [42]:
! ./pbrt-v3/build/pbrt pbrt-v3/scenes/example.pbrt

pbrt version 3 (built Jan 29 2023 at 15:40:14) [Detected 2 cores]
Copyright (c)1998-2018 Matt Pharr, Greg Humphreys, and Wenzel Jakob.
The source code to pbrt (but *not* the book contents) is covered by the BSD License.
See the file LICENSE.txt for the conditions of the license.
scenes/example.pbrt:5:55: Error: Unable to decode type from "lensradius"
scenes/example.pbrt:5:55: Warning: Type of parameter "lensradius" is unknown
scenes/example.pbrt:5:0: Error: Unable to decode type from "focaldistance"
scenes/example.pbrt:5:0: Warning: Type of parameter "focaldistance" is unknown
Rendering: [++]  (63.5s)       
Statistics:
  BVH
    Interior nodes                                                      1
    Leaf nodes                                                          2
    Primitives per leaf node                             3 /            2 (1.50x)
  Integrator
    Camera rays traced                                           20480000
    Path length                                  

In [31]:
%%file pbrt-v3/scenes/killeroo-simple1.pbrt 


LookAt 400 20 30   0 63 -110   0 0 1
Rotate -5 0 0 1
Camera "perspective" "float fov" [39]
Film "image"  
"integer xresolution" [512] "integer yresolution" [400]
    "string filename" "killeroo-simple1.png"

# zoom in by feet
# "integer xresolution" [1500] "integer yresolution" [1500]
#	"float cropwindow" [ .34 .49  .67 .8 ]

Sampler "halton" "integer pixelsamples" [8]

Integrator "path"

WorldBegin

AttributeBegin
Material "matte" "color Kd" [0 0 0]
Translate 150 0  20
Translate 0 120 0
AreaLightSource "area"  "color L" [2000 2000 2000] "integer nsamples" [8]
Shape "sphere" "float radius" [3]
AttributeEnd


AttributeBegin
  Material "matte" "color Kd" [.5 .5 .8]
  Translate 0 0 -140
Shape "trianglemesh" "point P" [ -1000 -1000 0 1000 -1000 0 1000 1000 0 -1000 1000 0 ]
      "float uv" [ 0 0 5 0 5 5 0 5 ]
	"integer indices" [ 0 1 2 2 3 0]
Shape "trianglemesh" "point P" [ -400 -1000 -1000   -400 1000 -1000   -400 1000 1000 -400 -1000 1000 ]
      "float uv" [ 0 0 5 0 5 5 0 5 ]
        "integer indices" [ 0 1 2 2 3 0]
AttributeEnd

AttributeBegin
Scale .5 .5 .5
Rotate -60 0 0 1
    Material "plastic" "color Kd" [.4 .2 .2] "color Ks" [.5 .5 .5]
        "float roughness" [.025]
#Translate -100 0 0
#    Include "geometry/killeroo.pbrt"
#    Material "plastic" "color Ks" [.3 .3 .3] "color Kd" [.4 .5 .4]
#        "float roughness" [.15] 
Translate -300 0 0
    Include "geometry/killeroo.pbrt"

AttributeEnd
WorldEnd


Overwriting scenes/killeroo-simple1.pbrt


In [ ]:
! ./pbrt-v3/build/pbrt pbrt-v3/scenes/killeroo-simple1.pbrt

# ISET RTF



In [12]:

import os
if os.path.exists('isetrtf'):
  print('Use local isetrtf repo.')
else:
  print('Checking out isetrtf repo ...')
  !git clone --recursive https://github.com/ISET/isetrtf.git

%ls

Use local isetrtf repo.
gdrive/  isetrtf/  pbrt-v3/  sample_data/


Display an PBRT screne from ISET_RTF to PBRT V3

In [36]:
%%file isetrtf/paper/data/scenes/stepfunction/stepfunction.pbrt

# PBRT file created with piWrite on 2022/3/4 9:47:28.42 
# PBRT version = 3 

LookAt 0.000000 0.000000 -2.000000 # eye 
      0.000000 0.000000 1.000000  # look at point
      0.000000 1.000000 0.000000  # up vector 

#Camera
Camera "perspective" "float fov" [90] # [31.417276] 
        "float focaldistance" [2000.000000] 
        "float lensradius" [0.000000] 
# sampler 
Sampler "halton" 
  "integer pixelsamples" [100] 

# film 
Film "image" "string filename" "stepfunction_pbrt.png" 
  "integer xresolution" [300] 
  "integer yresolution" [300] 
  "float diagonal" [0.900000] 

# integrator 
Integrator "path" 
  "integer maxdepth" [5] 

WorldBegin   

# uniform blue-ish illumination from all directions
#LightSource "infinite" "rgb L" [.4 .45 .5]

# approximate the sun
#LightSource "distant"  "point from" [ -30 40  100 ]
 #  "blackbody L" [3000 1.5]

AreaLightSource "diffuse" "rgb L" [0.5             0.5             0.5]

#AttributeBegin
  #Material "glass"
  #Shape "sphere" "float radius" 1
#AttributeEnd

AttributeBegin
  Translate 000 10.000 0.000
  # AreaLightSource "diffuse" "rgb L" [1             1             1]
  
  Shape "trianglemesh"  "integer indices" [0 1 2 2 3 0 ] "point P" [-100.0 -100.0 0    -100.0  100 0   0 100 0   0 -100 0 ]
AttributeEnd

WorldEnd

Overwriting isetrtf/paper/data/scenes/stepfunction/stepfunction.pbrt


In [37]:
! ./pbrt-v3/build/pbrt isetrtf/paper/data/scenes/stepfunction/stepfunction.pbrt

pbrt version 3 (built Jan 30 2023 at 20:07:43) [Detected 2 cores]
Copyright (c)1998-2018 Matt Pharr, Greg Humphreys, and Wenzel Jakob.
The source code to pbrt (but *not* the book contents) is covered by the BSD License.
See the file LICENSE.txt for the conditions of the license.
Rendering: [++]  (8.5s)       
Statistics:
  BVH
    Leaf nodes                                                          1
    Primitives per leaf node                             2 /            1 (2.00x)
  Integrator
    Camera rays traced                                            9000000
    Path length                                                     0.500 avg [range 0 - 1]
    Zero-radiance paths                            4500000 /      4500000 (100.00%)
  Intersections
    Regular ray intersection tests                               13500000
    Ray-triangle intersection tests                4500000 /     13500000 (33.33%)
  Memory
    BVH tree                                                         0